In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df_org = pd.read_csv('t20s_2013-23_mgd.csv').drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])

C:\Users\anees\AppData\Local\Temp\ipykernel_22248\3277827023.py:1: DtypeWarning: Columns (54,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df_org = pd.read_csv('t20s_2013-23_mgd.csv').drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])


In [3]:
df_org=df_org[(df_org['ball']<=11)]
df_org=df_org.drop(columns='ball.1')
df_org=df_org[(df_org['batruns']>=0) & (df_org['bowlruns']>=0)]

In [5]:
def bowl_style_clean(df,unique_styles):

    df['bowl_style'] = df['bowl_style'].replace(['RFM', 'RMF'], 'RFM')
    df['bowl_style'] = df['bowl_style'].replace(['LFM', 'LMF'], 'LFM')
    df['bowl_style'] = df['bowl_style'].replace(['LBG', 'LB'], 'RWS')
    df['bowl_style'] = df['bowl_style'].replace(['LS', 'SLA'], 'SLA')
    df['bowl_style'] = df['bowl_style'].replace(['RAB'], 'RSM')
    df['bowl_style'] = df['bowl_style'].replace(['LAB'], 'LSM')
    unique_styles = ['LFM/SLA', 'OB/LB','LM/SLA/LWS','OB/SLA','RMF/OB', 'RFM/OB', 'OB/LBG', 'RFM/LBG', 'RM/LB',
           'RM/LBG', 'LMF/RM', 'RM/RSM', 'SLA/LWS', 'RMF/LB', 'RFM/LB',
           'RM/OB/LB']
    replacements = {
        'RFM': 'RFM',
        'RMF': 'RFM',
        'LFM': 'LFM',
        'LMF': 'LFM',
        'LBG': 'RWS',
        'LB': 'RWS',
        'LS': 'SLA',
        'SLA': 'SLA',
        'RAB': 'RS',
        'LAB': 'LS'
    }

    # Iterate over unique styles and apply replacements
    for style in unique_styles:
        replaced_style = '/'.join([replacements.get(part, part) for part in style.split('/')])
        df['bowl_style'] = df['bowl_style'].replace([style], replaced_style)
    #df['bowl_style'].unique()
    return df
unique_styles = ['RFM', 'RMF', 'LBG', 'LB', 'LS', 'RAB', 'LAB', 'OB', 'SLA', 'RAB','RSM','LSM']
df=bowl_style_clean(df_org,unique_styles)
df['bowl_style'].unique()

array(['RF', 'RFM', 'RWS', 'LWS', 'SLA', 'OB', 'LFM', 'LF', 'RM', 'RM/OB',
       'LM', 'LFM/SLA', 'OB/RWS', 'LM/SLA/LWS', 'OB/SLA', 'RSM', '-',
       'LSM', 'RFM/OB', 'RFM/RWS', 'RM/RWS', 'LFM/RM', 'RM/RSM',
       'SLA/LWS', 'RM/OB/RWS', 'RS'], dtype=object)

In [7]:
df_worldcup = pd.read_csv('2024 wc squad.csv', encoding='latin1')
df_worldcup.head()

,Afghanistan,Australia,Bangladesh,Canada,England,India,Ireland,Namibia,Nepal,Netherlands,New Zealand,Oman,Papua New Guinea,Scotland,South Africa,Sri Lanka,Uganda,United States,West Indies,Pakistan
0,Rashid Khan,Mitchell Marsh,Najmul Hossain Shanto,Saad Bin Zafar,Jos Buttler,Rohit Sharma,Paul Stirling,Gerhard Erasmus,Rohit Paudel,Scott Edwards,Kane Williamson,Aqib Ilyas,Assad Vala,Richie Berrington,Aiden Markram,Wanindu Hasaranga,Brian Masaba,Monank Patel,Rovman Powell,NaN
1,Ibrahim Zadran,Ashton Agar,Taskin Ahmed,Aaron Johnson,Moeen Ali,Hardik Pandya,Mark Adair,Zane Green,Aasif Sheikh,Aryan Dutt,Finn Allen,Zeeshan Maqsood,Alei Nao,Matthew Cross,Ottniel Baartman,Charith Asalanka,Simon Ssesazi,Aaron Jones,Alzarri Joseph,NaN
2,Azmatullah Omarzai,Pat Cummins,Litton Das,Dillon Heyliger,Jofra Archer,Yashasvi Jaiswal,Ross Adair,Michael van Lingen,Anil Kumar Sah,Bas de Leede,Trent Boult,Kashyap Prajapati,Chad Soper,Bradley Currie,Gerald Coetzee,Kusal Mendis,Roger Mukasa,Andries Gous,Johnson Charles,NaN
3,Najibullah Zadran,Tim David,Soumya Sarkar,Dilpreet Bajwa,Jonny Bairstow,Virat Kohli,Andy Balbirnie,Dylan Leicher,Kushal Bhurtel,Daniel Doram,Michael Bracewell,Pratik Athavale,Charles Amini,Chris Greaves,Quinton de Kock,Pathum Nissanka,Cosmas Kyewuta,Corey Anderson,Roston Chase,NaN
4,Mohammad Ishaq,Nathan Ellis,Tanzid Hasan,Harsh Thaker,Harry Brook,Suryakumar Yadav,Curtis Campher,Ruben Trumpelmann,Kushal Malla,Fred Klaassen,Mark Chapman,Ayaan Khan,Hila Vare,Oli Carter,Bjorn Fortuin,Kamindu Mendis,Dinesh Nakrani,Ali Khan,Shimron Hetmyer,NaN


In [8]:
team_dict = df_worldcup.to_dict(orient='list')

print(team_dict['Afghanistan'])

['Rashid Khan', 'Ibrahim Zadran', 'Azmatullah Omarzai', 'Najibullah Zadran', 'Mohammad Ishaq', 'Mohammad Nabi', 'Gulbadin Naib', 'Karim Janat', 'Nangeyalia Kharote', 'Mujeeb Ur Rahman', 'Noor Ahmad', 'Naveen-ul-Haq', 'Fazalhaq Farooqi', 'Fareed Ahmad', nan, nan]


In [9]:
team_dict = {key: [value.strip() for value in values if pd.notna(value)] for key, values in team_dict.items()}

team_dict['Afghanistan']

['Rashid Khan',
 'Ibrahim Zadran',
 'Azmatullah Omarzai',
 'Najibullah Zadran',
 'Mohammad Ishaq',
 'Mohammad Nabi',
 'Gulbadin Naib',
 'Karim Janat',
 'Nangeyalia Kharote',
 'Mujeeb Ur Rahman',
 'Noor Ahmad',
 'Naveen-ul-Haq',
 'Fazalhaq Farooqi',
 'Fareed Ahmad']

In [10]:
players_list = np.concatenate((df['bowl'].unique(), df['bat'].unique()))
for team,players in team_dict.items():
    for player in players:
        if ((player not in players_list)|(player not in players_list)):
            print(team,player)

Canada Kanwarpal Tathgur
Namibia Jack Brassell
Namibia Malan Kruger
Namibia Peter-Daniel Blignaut
Nepal Anil Kumar Sah
Netherlands Daniel Doram
Netherlands Michael Levitt
Oman Khalid Kail
Scotland Jack Jarvis
Scotland Charlie Tear
United States Harmeet Singh
United States Nosthush Kenjige
United States Shayan Jahangir


In [5]:
relevant_competitions=['T20I', 'IPL', 'AUS v SL','T20 Blast','LPL', 'CPL', 'PSL', 'Big Bash League', 'BBL',
       "Men's Hundred",'MSL', 'MSL 2018','Super Smash','National T20 Cup', 'National T20', 'PAK-A v NZ-A T20s',
       'CSA T20 Challenge','CSA Provincial T20 Challenge', 'Syed Mushtaq Ali Trophy','SMA TROPHY','SMA TROPHY FINAL',
        'ENG v IND',  'CSA T20', 'Provincial T20','Africa T20 Cup',
       'PRO T20', 'Ireland A in Bangladesh unofficial T20I Series', 
       'Inter Pro T20', 'Hong Kong tour of Namibia','Ireland A in Namibia T20s', 'SA20',
       'PSL 2023', 'IPL 2023', 'CPL 2023']
df=df[df['competition'].isin(relevant_competitions)]

,Afghanistan,Australia,Bangladesh,Canada,England,India,Ireland,Namibia,Nepal,Netherlands,New Zealand,Oman,Papua New Guinea,Scotland,South Africa,Sri Lanka,Uganda,United States,West Indies,Pakistan
0,Rashid Khan,Mitchell Marsh,Najmul Hossain Shanto,Saad Bin Zafar,Jos Buttler,Rohit Sharma,Paul Stirling,Gerhard Erasmus,Rohit Paudel,Scott Edwards,Kane Williamson,Aqib Ilyas,Assadollah Vala,Richie Berrington,Aiden Markram,Wanindu Hasaranga,Brian Masaba,Monank Patel,Rovman Powell,NaN
1,Ibrahim Zadran,Ashton Agar,Taskin Ahmed,Aaron Johnson,Moeen Ali,Hardik Pandya,Mark Adair,Zane Green,Aasif Sheikh,Aryan Dutt,Finn Allen,Zeeshan Maqsood,Alei Nao,Matthew Cross,Ottniel Baartman,Charith Asalanka,Simon Ssesazi,Aaron Jones,Alzarri Joseph,NaN
2,Azmatullah Omarzai,Pat Cummins,Litton Das,Dilon Heyliger,Jofra Archer,Yashasvi Jaiswal,Ross Adair,Michael Van Lingen,Anil Kumar Sah,Bas de Leede,Trent Boult,Kashyap Prajapati,Chad Soper,Brad Currie,Gerald Coetzee,Kusal Mendis,Roger Mukasa,Andries Gous,Johnson Charles,NaN
3,Najibullah Zadran,Tim David,Soumya Sarkar,Dilpreet Bajwa,Jonathan Bairstow,Virat Kohli,Andrew Balbirnie,Dylan Leicher,Kushal Bhurtel,Daniel Doram,Michael Bracewell,Pratik Athavale,CJ Amini,Chris Greaves,Quinton de Kock,Pathum Nissanka,Cosmas Kyewuta,Corey Anderson,Roston Chase,NaN
4,Mohammad Ishaq,Nathan Ellis,Tanzid Hasan Tamim,Harsh Thaker,Harry Brook,Suryakumar Yadav,Curtis Campher,Ruben Trumpelmann,Kushal Malla,Fred Klaassen,Mark Chapman,Ayaan Khan,Hila Vare,Oli Hairs,Bjorn Fortuin,Kamindu Mendis,Dinesh Nakrani,Ali Khan,Shimron Hetmyer,NaN


In [44]:
players_list = np.concatenate((df['bowl'].unique(), df['bat'].unique()))
for team,players in team_dict.items():
    for player in players:
        if ((player not in players_list)|(player not in players_list)):
            print(team,player)
#df[df['bowl']=='Mohammad Siraj']

Afghanistan Mohammad Ishaq
Afghanistan Nangyal Kharoti
Afghanistan Fareed Ahmad Malik
Bangladesh Tanzid Hasan Tamim
Bangladesh Tawhid Hridoy
Bangladesh Mahmud Ullah Riyad
Bangladesh Jaker Ali Anik
Bangladesh Shak Mahedi Hasan
Canada Dilon Heyliger
Canada Kanwarpal Tathgur
Canada Rayyankhan Pathan
England  Jonathan Bairstow
India Mohd. Siraj
Ireland Andrew Balbirnie
Namibia Michael Van Lingen
Namibia Jack Brassell
Namibia Niko Davin
Namibia Malan Kruger
Namibia PD Blignaut
Nepal Anil Kumar Sah
Nepal Gulshan Jha
Nepal Kamal Singh Airee
Netherlands Daniel Doram
Netherlands Michael Levitt
Netherlands ??Tim Pringle
Netherlands Vikram Singh
Netherlands Viv Kingma
New Zealand Jimmy Neesham
Oman Shakeel Ahmad
Oman Khalid Kail
Papua New Guinea Assadollah Vala
Papua New Guinea CJ Amini
Papua New Guinea Kabua Vagi Morea
Papua New Guinea Kipling Doriga
Papua New Guinea Sema Kamea
Scotland Brad Currie
Scotland Oli Hairs
Scotland Jack Jarvis
Scotland Charlie Tear
Sri Lanka Dhananjaya De Silva
Uganda